In [0]:




# Databricks Notebook: Komplett ETL-pipeline for parkeringsdata (uten Unity Catalog)

# ------------------------------
# 1. Hent data fra ekstern URL
# ------------------------------
import requests
import json
from pyspark.sql import SparkSession
from tests.etl_pipeline import sjekk_duplikater, valider_manglende, valider_gyldige_verdier, konverter_timestamp

# Hent JSON-data fra en offentlig URL
url = "https://opencom.no/dataset/36ceda99-bbc3-4909-bc52-b05a6d634b3f/resource/d1bdc6eb-9b49-4f24-89c2-ab9f5ce2acce/download/parking.json"
response = requests.get(url)
data = response.json()

# Konverter JSON-data til Spark DataFrame
# (Datatypene blir automatisk inferert, men kan tilpasses manuelt ved behov)
df_raw = spark.createDataFrame(data)




In [0]:
# ----------------------------------------------------
# 2. Kvalitetssikre data og konverter dato/klokkeslett
# ----------------------------------------------------


valider_manglende(df_raw) # sjekk at det ikke finnes manglende verdier å rådataene 
valider_gyldige_verdier(df_raw) # sjekk at verdiene er gyldige 
df_deduped = sjekk_duplikater(df_raw) # sjekk for duplikater
df_cleaned = konverter_timestamp(df_deduped) # rens og konverter dato og klokkelett


# df_cleaned er nå klar til å bli lagret i silver



In [0]:
# 3. Lagre staging-tabell (bronse)
# ----------------------------------
# Append for å bevare historiske data (bronse = ubehandlet rådata)
bronze_path = "/mnt/bronze/staging_parking"
df_raw.write.format("delta").mode("append").option("mergeSchema", "true").save(bronze_path)
spark.sql(f"CREATE TABLE IF NOT EXISTS default.staging_parking USING DELTA LOCATION '{bronze_path}'")


In [0]:
# ----------------------------------
# 4. Dimensjonstabell: Parkering
# ----------------------------------
from pyspark.sql.functions import col

# Hent eksisterende tabell hvis den finnes, ellers None
existing_dim_parkering = spark.table("default.dim_parkering") if "default.dim_parkering" in [t.name for t in spark.catalog.listTables("default")] else None

# Finn nye unike parkeringsplasser og gi kolonnene mer beskrivende navn
new_dim_parkering = df_cleaned.select("Sted", "Latitude", "Longitude").distinct()
new_dim_parkering = new_dim_parkering.withColumnRenamed("Sted", "Parkering_navn")

# Vi antar at det alltid skal være nøyaktig 9 parkeringsplasser. Skriv bare hvis antallet ikke stemmer.
if existing_dim_parkering is None or existing_dim_parkering.count() != 9:
    new_dim_parkering.write.format("delta").mode("overwrite").saveAsTable("default.dim_parkering")

In [0]:
# ----------------------------------
# 5. Dimensjonstabell: Tid
# ----------------------------------



from pyspark.sql.functions import to_date, hour, minute
from pyspark.sql import Row

# Create an empty DataFrame with the same schema
#empty_df = spark.createDataFrame([], spark.table("default.dim_tid").schema)

# Overwrite the table with empty data
#empty_df.write.mode("overwrite").saveAsTable("default.dim_tid")


# Bygg batch kun basert på nyeste data
dim_tid_batch = df_cleaned.select(
    to_date("timestamp").cast("date").alias("dato"),
    hour("timestamp").cast("int").alias("time"),
    minute("timestamp").cast("int").alias("minutt")
).distinct()

# Sjekk om tabellen finnes
if "dim_tid" in [t.name for t in spark.catalog.listTables("default")]:
    # Hent eksisterende tabell
    existing_dim_tid = spark.table("default.dim_tid").select("dato", "time", "minutt")

    # Finn tidspunkt som ikke finnes fra før
    delta_tid = dim_tid_batch.join(existing_dim_tid, ["dato", "time", "minutt"], how="left_anti")

    # Legg til nye rader hvis noen er nye
    if delta_tid.count() > 0:
        delta_tid.write.format("delta").mode("append").saveAsTable("default.dim_tid")
else:
    # Hvis tabellen ikke finnes, opprett den
    dim_tid_batch.write.format("delta").mode("overwrite").saveAsTable("default.dim_tid")


In [0]:
# ----------------------------------
# 6. Faktatabell: Parkeringskapasitet
# ----------------------------------
from pyspark.sql.functions import to_date, hour, minute, date_format, to_timestamp, when, lit
from pyspark.sql.functions import col, max as max_

# Hent 9 observasjoner per kjøring (én per lokasjon)

# spark.sql("TRUNCATE TABLE fakt_parkering")

new_fakt = df_cleaned.select(
    to_date("timestamp").alias("dato"),
    date_format(to_timestamp("timestamp"), "HH:mm").alias("klokkeslett"),  
    "Sted",
    "Antall_ledige_plasser",
    "timestamp"
)


# Append for å lagre hver kjørings observasjoner
new_fakt.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("default.fakt_parkering")

# Les tabellen på nytt, sorter og lagre tilbake
df_sorted = spark.table("default.fakt_parkering") \
    .orderBy("timestamp", "Sted")

df_sorted.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("default.fakt_parkering")

# fakt_parkering er nå gold-layer


# Les eksisterende tabell, hent de 9 siste og sorter deretter
latest9 = spark.table("default.fakt_parkering") \
    .orderBy(col("timestamp").desc()) \
    .limit(9) \
    .orderBy(col("Antall_ledige_plasser").asc())


# Lagre som ny tabell
latest9.write.format("delta").mode("overwrite").saveAsTable("default.fakt_parkering_siste9")





In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`fakt_parkering`

In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`fakt_parkering_siste9` limit 100;